In [1]:
import sys
import copy 
import os
import psutil
import time
import math
method=sys.argv[0]
state=sys.argv[1]
driver_3(method,state)

NameError: name 'driver_3' is not defined

In [197]:
def to2D(state1D): # returns 2D representation of 1D state
    return [[state1D[i+j*N] for i in range(N)] for j in range(N)]
def to1D(state2D): # returns 1D state from 2D representation
    return [item for sublist in state2D for item in sublist]

In [198]:
def get_children(state): # returns list of 1D state of children UDLR ordered
    state2D = to2D(state)
    idx = state.index(0)
    idx2D = divmod(idx,N)
    children=[None]*4
    if (idx2D[0]>0): 
        child2D = copy.deepcopy(state2D)
        child2D[idx2D[0]][idx2D[1]] = state2D[idx2D[0]-1][idx2D[1]]
        child2D[idx2D[0]-1][idx2D[1]] = 0
        children[0]=to1D(child2D)
    if (idx2D[0]<N-1): 
        child2D = copy.deepcopy(state2D)
        child2D[idx2D[0]][idx2D[1]] = state2D[idx2D[0]+1][idx2D[1]]
        child2D[idx2D[0]+1][idx2D[1]] = 0
        children[1]=to1D(child2D)
    if (idx2D[1]>0): 
        child2D = copy.deepcopy(state2D)
        child2D[idx2D[0]][idx2D[1]] = state2D[idx2D[0]][idx2D[1]-1]
        child2D[idx2D[0]][idx2D[1]-1] = 0
        children[2]=to1D(child2D)
    if (idx2D[1]<N-1): 
        child2D = copy.deepcopy(state2D)
        child2D[idx2D[0]][idx2D[1]] = state2D[idx2D[0]][idx2D[1]+1]
        child2D[idx2D[0]][idx2D[1]+1] = 0
        children[3]=to1D(child2D)
    return children

In [199]:
def get_man(state):
    state2D=to2D(state)
    dist=0
    for i in range(1,N**2):
        idx = state.index(i)
        idx2D = divmod(idx,N)
        target=divmod(i,N)
        dist=dist+abs(idx2D[0]-target[0])+abs(idx2D[1]-target[1])
    return dist

In [200]:
class State:
    def set_loc(self, new_loc):
        self.loc = new_loc #state in 1D list
    def set_parent(self, new_parent):
        self.parent = new_parent #parent of class 'state'
    def set_depth(self, new_depth): # depth of state in tree
        self.depth = new_depth
    def get_stamp(self): #returns string stamp of 'self.loc'
        return "".join(str(x) for x in self.loc)
    def get_path(self): #builds the path to this state reading 'self.parent' recursively
        st = self
        path = []
        while st.parent!=None:
            path.append(st.dir)
            st = st.parent
        return list(reversed(path))
    def set_dir(self, new_dir): #set last direction to get here, 0 to 3
        self.dir =new_dir
    def check(self): #check if solution reached
        return (list(range(N**2)) == self.loc)

In [201]:
def BFS(initialState):
    process = psutil.Process(os.getpid())
    frontier = []
    state=State()
    state.set_loc(initialState)
    state.set_depth(0)
    state.set_parent(None)
    frontier.append(state)
#    explored = set()
    exploredstamps= set()
    UDLR=["Up","Down","Left","Right"]
    max_fringe_size=0
    max_depth=0
    max_mem=0
    while not(frontier==[]): 
        max_fringe_size=max(max_fringe_size,len(frontier))
        max_mem=max(max_mem,process.memory_info().rss/(1024*1024))
        state = frontier.pop(0)
        if (state.check()): return [[UDLR[i] for i in state.get_path()],len(state.get_path()), 
                                    len(exploredstamps)-len(frontier)-1, len(frontier), max_fringe_size, 
                                    state.depth, max_depth,time.time(),max_mem]
        for index,item in enumerate(get_children(state.loc)):
            if item!=None:
                child=State()
                child.set_loc(item)
                if not(child.get_stamp() in exploredstamps):
                    child.set_depth(state.depth+1)
                    child.set_parent(state)
                    child.set_dir(index)
                    frontier.append(child)
#                    explored.add(child)
                    exploredstamps.add(child.get_stamp())
                    max_depth=max(max_depth,child.depth)
    return False

In [211]:
def DFS(initialState):
    process = psutil.Process(os.getpid())
    frontier = []
    state=State()
    state.set_loc(initialState)
    state.set_depth(0)
    state.set_parent(None)
    frontier.append(state)
#    explored = set()
    exploredstamps= set()
    UDLR=list(reversed(["Up","Down","Left","Right"]))
    max_fringe_size=0
    max_depth=0
    max_mem=0
    while not(frontier==[]): 
        max_fringe_size=max(max_fringe_size,len(frontier))
        max_mem=max(max_mem,process.memory_info().rss/(1024*1024))
        state = frontier.pop()
        if (state.check()): return [[UDLR[i] for i in state.get_path()],len(state.get_path()), 
                                    len(exploredstamps)-len(frontier)-1, len(frontier), max_fringe_size, 
                                    state.depth, max_depth,time.time(),max_mem]
        for index,item in enumerate(list(reversed(get_children(state.loc)))):
            if item!=None:
                child=State()
                child.set_loc(item)
                if not(child.get_stamp() in exploredstamps):
                    child.set_depth(state.depth+1)
                    child.set_parent(state)
                    child.set_dir(index)
                    frontier.append(child)
#                    explored.add(child)
                    exploredstamps.add(child.get_stamp())
                    max_depth=max(max_depth,child.depth)
    return False

In [217]:
def getMinCost(frontier,frontierkeys):
    mins=[i for i, x in enumerate(frontierkeys) if x == min(frontierkeys)] #get all indexes with min cost
#    print(mins)
    if (len(mins)>1):
        indexes=[]
        for i in range(4):
            for j in mins:
                if frontier[j].dir==i: indexes.append(j)
            if len(indexes)>0: return indexes[0]
            indexes=[]
    return mins[0]

In [221]:
def AST(initialState):
    process = psutil.Process(os.getpid())
    frontier = []
    frontierkeys = []
    state=State()
    state.set_loc(initialState)
    state.set_depth(0)
    state.set_parent(None)
    frontier.append(state)
    frontierkeys.append(0)
#    explored = set()
    exploredstamps= set()
    UDLR=["Up","Down","Left","Right"]
    max_fringe_size=0
    max_depth=0
    max_mem=0
    while not(frontier==[]): 
        max_fringe_size=max(max_fringe_size,len(frontier))
        max_mem=max(max_mem,process.memory_info().rss/(1024*1024))
        index=getMinCost(frontier,frontierkeys)
        state = frontier.pop(index)
        frontierkeys.pop(index)
        if (state.check()): return [[UDLR[i] for i in state.get_path()],len(state.get_path()), 
                                    len(exploredstamps)-len(frontier)-1, len(frontier), max_fringe_size, 
                                    state.depth, max_depth,time.time(),max_mem]
        for index,item in enumerate(get_children(state.loc)):
            if item!=None:
                child=State()
                child.set_loc(item)
                if not(child.get_stamp() in exploredstamps):
                    child.set_depth(state.depth+1)
                    child.set_parent(state)
                    child.set_dir(index)
                    frontier.append(child)
                    frontierkeys.append(child.depth+get_man(child.loc))
#                    explored.add(child)
                    exploredstamps.add(child.get_stamp())
                    max_depth=max(max_depth,child.depth)
    return False

In [269]:
def IDA(initialState):
    process = psutil.Process(os.getpid())
    d=0
    max_fringe_size=0
    max_depth=0
    max_mem=0
    UDLR=list(reversed(["Up","Down","Left","Right"])) 
    while(True):
        d=d+1
        frontier = []
        frontierkeys = []
        state=State()
        state.set_loc(initialState)
        state.set_depth(0)
        state.set_parent(None)
        frontier.append(state)
        frontierkeys.append(0)
    #    explored = set()
        exploredstamps= set()
        while not(frontier==[]): 
            max_fringe_size=max(max_fringe_size,len(frontier))
            max_mem=max(max_mem,process.memory_info().rss/(1024*1024))
            index=getMinCost(frontier,frontierkeys)
            state = frontier.pop(index)
            frontierkeys.pop(index)
            if (state.check()): return [[UDLR[i] for i in state.get_path()],len(state.get_path()), 
                                        len(exploredstamps)-len(frontier)-1, len(frontier), max_fringe_size, 
                                        state.depth, max_depth,time.time(),max_mem]
            if (state.depth<d):    
                for index,item in enumerate(list(reversed(get_children(state.loc)))):
                    if item!=None:
                        child=State()
                        child.set_loc(item)
                        if not(child.get_stamp() in exploredstamps):
                            child.set_depth(state.depth+1)
                            child.set_parent(state)
                            child.set_dir(index)
                            frontier.append(child)
                            frontierkeys.append(child.depth+get_man(child.loc))
        #                    explored.add(child)
                            exploredstamps.add(child.get_stamp())
                            max_depth=max(max_depth,child.depth)
    return False

In [301]:
def solve(method,state):
    N=int(math.sqrt(len(state)))
    options = {"bfs" : BFS,
           "dfs" : DFS,
           "ast" : AST,
           "ida" : IDA,
    }
    process = psutil.Process(os.getpid())
    baseMem= process.memory_info().rss/(1024*1024)
    baseTime=time.time()
    solved=options[method](state)
    solved[7]="{0:.8f}".format(solved[7]-baseTime)
    solved[8]="{0:.8f}".format(solved[8]-baseMem)
    headers=["path_to_goal: ","cost_of_path: ","nodes_expanded: ","fringe_size: ","max_fringe_size: ",
             "search_depth: ","max_search_depth: ","running_time: ","max_ram_usage: "]
    values=[]
    for i,j in zip(headers,solved):
        values.append(i+str(j))
    text_file = open("output.txt", "w")
    text_file.write("\n".join(values))
    text_file.close()

In [58]:
s.set_dir(s1)

In [59]:
p=State()

In [60]:
s.set_parent(None)

In [61]:
p.set_parent(s)

In [62]:
p.set_dir(1)

In [63]:
p.get_path()

[1]

In [9]:
a.pop()

4

In [10]:
a

[1, 2, 3]

In [319]:
a

{1, 2}

In [155]:
UDLR=["U","D","L","R"]

In [236]:
s2 = [1,2,5,3,4,0,6,7,8]

In [165]:
b

[3, 4]

In [166]:
b.append(5)

In [170]:
b.pop(0)

3

In [73]:
a.path

[2]

In [127]:
s1=[1, 2, 0, 3, 4, 5, 6, 7, 8]

In [ ]:
s1

In [153]:
st1.children

[None, [3, 1, 2, 0, 4, 5, 6, 7, 8], None, [1, 0, 2, 3, 4, 5, 6, 7, 8]]

In [87]:
list(range(N**2))

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [95]:
divmod(3,3)

(1, 0)

In [193]:
s1=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [194]:
get_man(s1)

0

In [101]:
[None]*4

[None, None, None, None]

In [98]:
[s1,s1,None]

[[0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8], None]

In [25]:
t1[0][3]

IndexError: list index out of range